In [11]:
import sys
sys.path.append('../reconstruct_missing_data')

from data_loading import find_data_files, load_data_set, get_anomalies, clone_data, create_missing_mask, split_and_scale_data

import numpy as np
import xarray as xr
from pathlib import Path
from json import dump, load
import os

In [2]:
# Check current working directory:
print(os.getcwd())

/gxfs_work1/geomar/smomw511/GitHub/MarcoLandtHayen/reconstruct_missing_data/notebooks


In [3]:
### Set parameters up-front:

## Decide to work on test data or full data:
path_to_data = '../data/test_data/' # Test data
# path_to_data = '../../../../climate_index_collection/data/raw/2022-08-22/' # Full data

# Model configuration, to store results:
model_config = 'unet_4conv'

# Data loading and preprocessing:
feature = 'sea-level-pressure' # Choose either 'sea-level-pressure' or 'sea-surface-temperature' as feature.
feature_short = 'slp' # Free to set short name, to store results, e.g. 'slp' and 'sst'.
source = 'FOCI' # Choose Earth System Model, either 'FOCI' or 'CESM'.
mask_type = 'fixed' # Can have random missing values, individually for each data sample ('variable'), 
                    # or randomly create only a single mask, that is then applied to all samples identically ('fixed').
missing_type = 'discrete' # Either specify discrete amounts of missing values ('discrete') or give a range ('range').
augmentation_factor = 1 # Number of times, each sample is to be cloned, keeping the original order.
train_val_split = 0.8 # Set rel. amount of samples used for training.
missing_values = [0.99, 0.95, 0.9, 0.75, 0.5] # Set array for desired amounts of missing values: 0.9 means, that 90% of the values are missing.
                                              # Or set a range by only giving minimum and maximum allowed relative amounts of missing values, 
                                              # e.g. [0.75, 0.95], according to missing_type 'discrete' or 'range', respectively.
scale_to = 'zero_one' # Choose to scale inputs to [-1,1] ('one_one') or [0,1] ('zero_one') or 'norm' to normalize inputs or 'no' scaling.

# To build, compile and train model:
CNN_filters = [64,128,256,512]# [2,4,8,16] # Number of filters.
CNN_kernel_size = 5 # Kernel size
learning_rate = 0.0005
loss_function = 'mse' 
epochs = 10
batch_size = 10

In [10]:
# Create directory to store results: Raise error, if path already exists, to avoid overwriting existing results. 
path = Path('../../../../GitGeomar/marco-landt-hayen/reconstruct_missing_data/results/'+model_config+'_'+feature_short+'_'+source+'_'+mask_type+'_'+missing_type+'_factor_'+str(augmentation_factor))
os.makedirs(path, exist_ok=False)
    
# Store parameters as json:
parameters = {
    'model_config': model_config,
    'feature': feature,
    'feature_short': feature_short,
    'source': source,
    'mask_type': mask_type,
    'missing_type': missing_type,
    'augmentation_factor': augmentation_factor,
    'train_val_split': train_val_split,
    'missing_values': missing_values,
    'scale_to': scale_to,
    'CNN_filters': CNN_filters,
    'CNN_kernel_size': CNN_kernel_size,
    'learning_rate': learning_rate,
    'loss_function': loss_function,
    'epochs': epochs,
    'batch_size': batch_size
}

with open(path / 'parameters.json', 'w') as f:
    dump(parameters, f)

In [9]:
# Look for data files:
find_data_files(data_path=path_to_data, data_source_name=source)

[PosixPath('../data/test_data/FOCI/FOCI1.3-SW038_1m_23500101_23591231_grid_T_atmos_grid.nc'),
 PosixPath('../data/test_data/FOCI/FOCI1.3-SW038_echam6_ATM_mm_2350-2359_geopoth_pl_monthly_50000.nc'),
 PosixPath('../data/test_data/FOCI/FOCI1.3-SW038_echam6_BOT_mm_2350-2359_precip_monthly_1.nc'),
 PosixPath('../data/test_data/FOCI/FOCI1.3-SW038_echam6_BOT_mm_2350-2359_slp_monthly_1.nc'),
 PosixPath('../data/test_data/FOCI/FOCI1.3-SW038_echam6_BOT_mm_2350-2359_temp2_monthly_1.nc'),
 PosixPath('../data/test_data/FOCI/FOCI1.3-SW038_echam6_BOT_mm_2350-2359_tsw_monthly_1.nc')]

In [4]:
# Load data, including ALL fields and mask for Ocean values:
data = load_data_set(data_path=path_to_data, data_source_name=source)
#data

/app/conda/lib/python3.8/site-packages/xarray/coding/times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/app/conda/lib/python3.8/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


In [5]:
# Select single feature and compute anomalies, using whole time span as climatology:
data = get_anomalies(feature=feature, data_set=data)

In [6]:
# Extend data, if desired:
data_temp = clone_data(data=data, augmentation_factor=augmentation_factor)

In [7]:
missing_min=missing_values[0]
missing_max=missing_values[0]

In [8]:
# Create mask for missing values:
missing_mask = create_missing_mask(data=data, mask_type=mask_type, missing_type=missing_type, missing_min=missing_min, missing_max=missing_max)

In [9]:
np.sum(missing_mask,axis=(1,2))

array([183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183,
       183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183,
       183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183,
       183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183,
       183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183,
       183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183,
       183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183,
       183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183,
       183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183,
       183, 183, 183])

In [12]:
# Use sparse data as inputs and complete data as targets. Split sparse and complete data into training and validation sets. 
# Scale or normlalize data according to statistics obtained from only training data.
train_input, val_input, train_target, val_target, train_min, train_max, train_mean, train_std = split_and_scale_data(
    data, 
    missing_mask, 
    train_val_split, 
    scale_to
)

In [ ]:
## Train models:

# Loop over array of desired sparsity:
for i in range(len(sparsity_all)):
    
    # Get current sparsity:
    sparsity = sparsity_all[i]
    
    # Print status:
    print("Sparsity: ", i+1, " of ", len(sparsity_all))
    
    # Create sub-directory to store results: Raise error, if path already exists, to avoid overwriting existing results.
    os.makedirs(path / 'sparsity_' f'{int(sparsity*100)}', exist_ok=False)
    
    # Load data:
    data = load_data(source=source)
    
    # Extend data, if desired:
    data = clone_data(data=data, augmentation_factor=augmentation_factor)
    
    # Create sparsity mask:
    sparsity_mask = create_sparsity_mask(data=data, sparsity=sparsity, mask_type=mask_type)
    
    # Store sparsity mask:
    np.save(path / 'sparsity_' f'{int(sparsity*100)}' / 'sparsity_mask.npy', sparsity_mask)

    # Use sparse data as inputs and complete data as targets. Split sparse and complete data into training and validation sets. 
    # Scale or normlalize data according to statistics obtained from only training data.
    train_input, val_input, train_target, val_target, train_min, train_max, train_mean, train_std = split_and_scale_data(
        data, 
        missing_mask, 
        train_val_split, 
        scale_to
    )
    
    # Build and compile U-Net model:
    model = build_unet_4conv(input_shape=(train_input.shape[1],train_input.shape[2],1),
                         CNN_filters=CNN_filters,
                         CNN_kernel_size=CNN_kernel_size,
                         learning_rate=learning_rate,
                         loss_function=loss_function,
                        )
    
    # Save untrained model:
    model.save(path / 'sparsity_' f'{int(sparsity*100)}' / f'epoch_{0}')
    
    # Initialize storage for training and validation loss:
    train_loss=[]
    val_loss=[]
    
    # Get model predictions on train and validation data FROM UNTRAINED MODEL!
    train_pred = model.predict(train_input)
    val_pred = model.predict(val_input)
    
    # Store loss on training and validation data:
    train_loss.append(np.mean((train_pred[:,:,:,0]-train_target)**2))
    val_loss.append(np.mean((val_pred[:,:,:,0]-val_target)**2))
    
    # Loop over number of training epochs:
    for j in range(epochs):
        
        # Print status:
        print("  Epoch: ", j+1, " of ", epochs)
        
        # Train model on sparse inputs with complete 2D fields as targets, for SINGLE epoch:
        history = model.fit(train_input, train_target, epochs=1, verbose=0, shuffle=True,
                            batch_size=batch_size, validation_data=(val_input, val_target))
        
        # Save trained model after current epoch:
        model.save(path / 'sparsity_' f'{int(sparsity*100)}' / f'epoch_{j+1}')
        
        # Get model predictions on train and validation data AFTER current epoch:
        train_pred = model.predict(train_input)
        val_pred = model.predict(val_input)

        # Store loss on training and validation data:
        train_loss.append(np.mean((train_pred[:,:,:,0]-train_target)**2))
        val_loss.append(np.mean((val_pred[:,:,:,0]-val_target)**2))  
        
    # Save loss:
    np.save(path / 'sparsity_' f'{int(sparsity*100)}' / 'train_loss.npy', train_loss)
    np.save(path / 'sparsity_' f'{int(sparsity*100)}' / 'val_loss.npy', val_loss)

In [16]:
# Extract single field, here: Sea level pressure
slp_FOCI = data_FOCI['sea-level-pressure'].values
slp_FOCI.shape

(12000, 96, 192)